In [23]:
import pandas_datareader.data as web
import datetime

start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2021, 8, 11)
df = web.DataReader('GOOGL', 'stooq', start, end)

In [24]:
def Stock_Price_LSTM_Data_Precesing(df, mem_his_days, pre_days):
    df.dropna(inplace=True)
    df.sort_index(inplace=True)  # 按时间排序

    df['label'] = df['Close'].shift(-pre_days)  # 收盘价前移10天作为label

    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler()
    sca_X = scaler.fit_transform(df.iloc[:, :-1])  # 标准化(不包括最后一列label)

    from collections import deque

    deq = deque(maxlen=mem_his_days)  # 设定队列, 最大长度为记忆天数

    X = []
    for i in sca_X:
        deq.append(list(i))
        if len(deq) == mem_his_days:
            X.append(list(deq))

    X_lately = X[-pre_days:]
    X = X[:-pre_days]  # 删除最后几行(预测天数), 因为没有label

    y = df['label'].values[mem_his_days - 1:-pre_days]

    import numpy as np

    X = np.array(X)
    y = np.array(y)

    return X, y, X_lately

In [25]:
mem_his_days = 10  # 记忆天数: 10天
pre_days = 10  # 预测10天后

In [26]:
X, y, X_lately = Stock_Price_LSTM_Data_Precesing(df, 10, 10)

In [27]:
print(len(X))
print(len(y))
print(len(X_lately))

4255
4255
10


In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

构建lstm

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
# 构建第一层
model.add(LSTM(10, input_shape=X.shape[1:], activation='tanh', return_sequences=True))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建第二层
model.add(LSTM(10, activation='tanh', return_sequences=True))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建第三层
model.add(LSTM(10, activation='tanh'))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建全连接层
model.add(Dense(10, activation='tanh'))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 输出层
model.add(Dense(1))

# 编译
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mape'])

# 训练模型
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


2022-11-25 20:53:03.122854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:03.289782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:03.574007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:03.619851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:03.718875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  1/120 [..............................] - ETA: 4:59 - loss: 1162.5134 - mape: 100.2654

2022-11-25 20:53:03.897219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:03.969810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - ETA: 0s - loss: 1456.6760 - mape: 85.6720

2022-11-25 20:53:06.887782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:06.949818: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:06.986563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-25 20:53:07.023424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 6s 27ms/step - loss: 1456.6760 - mape: 85.6720 - val_loss: 1246.0389 - val_mape: 67.6646
Epoch 2/50
120/120 [==============================] - 2s 18ms/step - loss: 1255.5714 - mape: 64.6082 - val_loss: 1161.1906 - val_mape: 61.4738
Epoch 3/50
120/120 [==============================] - 2s 19ms/step - loss: 1180.2516 - mape: 58.7051 - val_loss: 1098.0509 - val_mape: 57.8215
Epoch 4/50
120/120 [==============================] - 2s 19ms/step - loss: 1121.6647 - mape: 55.0075 - val_loss: 1043.6600 - val_mape: 55.5319
Epoch 5/50
120/120 [==============================] - 2s 19ms/step - loss: 1066.6799 - mape: 52.7469 - val_loss: 995.7532 - val_mape: 54.6889
Epoch 6/50
120/120 [==============================] - 3s 21ms/step - loss: 1018.7001 - mape: 51.8178 - val_loss: 952.7134 - val_mape: 55.0045
Epoch 7/50
120/120 [==============================] - 2s 20ms/step - loss: 978.1583 - mape: 52.9962 - val_loss: 913.9058 - val_mape: 56.4265
Epoch 8/50
120

In [68]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
# 构建第一层
model.add(LSTM(10, input_shape=X.shape[1:], activation='relu', return_sequences=True))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建第二层
model.add(LSTM(10, activation='relu', return_sequences=True))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建第三层
model.add(LSTM(10, activation='relu'))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 构建全连接层
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))  # 为防止过拟合, 删除0.1%的神经元

# 输出层
model.add(Dense(1))

# 编译
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mape'])

# 训练模型
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50


2022-11-25 16:38:54.942341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - ETA: 0s - loss: 1156.0809 - mape: 87.4772

2022-11-25 16:39:10.090685: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


120/120 [==============================] - 17s 127ms/step - loss: 1156.0809 - mape: 87.4772 - val_loss: 379.7974 - val_mape: 55.6922
Epoch 2/50
120/120 [==============================] - 14s 117ms/step - loss: 184.1381 - mape: 37.0181 - val_loss: 35.2082 - val_mape: 14.2493
Epoch 3/50
120/120 [==============================] - 14s 116ms/step - loss: 91.1165 - mape: 22.7248 - val_loss: 31.7139 - val_mape: 12.7926
Epoch 4/50
120/120 [==============================] - 15s 121ms/step - loss: 71.7090 - mape: 20.9853 - val_loss: 31.9519 - val_mape: 12.0242
Epoch 5/50
120/120 [==============================] - 14s 118ms/step - loss: 53.8750 - mape: 19.1681 - val_loss: 39.0508 - val_mape: 12.9677
Epoch 6/50
120/120 [==============================] - 13s 111ms/step - loss: 46.6513 - mape: 18.4253 - val_loss: 50.1344 - val_mape: 12.5182
Epoch 7/50
120/120 [==============================] - 13s 109ms/step - loss: 38.6148 - mape: 17.5123 - val_loss: 52.1435 - val_mape: 13.7638
Epoch 8/50
120/120 [